In [1]:
import tensorflow as tf
import numpy as np
import re

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
word2idx = {'PAD': 0,'NUM':1,'UNK':2}
tag2idx = {'PAD': 0}
char2idx = {'PAD': 0}
word_idx = 3
tag_idx = 1
char_idx = 1

In [3]:
def process_word(word, lower=False):
    if lower:
        word = word.lower()
    if word.isdigit():
        word = 'NUM'
    else:
        word = re.sub('[^A-Za-z0-9\- ]+', '', word)
    return word

def read_file(file):
    global word_idx, tag_idx, char_idx
    with open(file,'r') as f:
        words, tags, X, Y = [], [], [], []
        for line in f:
            line = line.strip()
            if (len(line) == 0 or line.startswith("-DOCSTART-")):
                continue
            else:
                ls = line.split(' ')
                if len(ls) > 1:
                    word, tag = ls[0],ls[-1]
                else:
                    word = ls[0]
                    tag = 'O'
                for c in word:
                    if c not in char2idx:
                        char2idx[c] = char_idx
                        char_idx += 1
                word = process_word(word,True)
                if len(word) < 1:
                    continue
                words += [word]
                tags += [tag]
                if word not in word2idx:
                    word2idx[word] = word_idx
                    word_idx += 1
                X.append(word2idx[word])
                if tag not in tag2idx:
                    tag2idx[tag] = tag_idx
                    tag_idx += 1
                Y.append(tag2idx[tag])
                        
    return words, tags, X, Y

In [4]:
train_words, train_tags, train_X, train_Y = read_file('eng.train')

In [5]:
test_words, test_tags, test_X, test_Y = read_file('eng.testa')

In [6]:
idx2tag={idx: tag for tag, idx in tag2idx.items()}
idx2word={idx: tag for tag, idx in word2idx.items()}
batch_size = 64
dim_word = 128
dim_char = 32
dropout = 0.8
learning_rate = 1e-3
hidden_size_char = 64
hidden_size_word = 128
num_layers = 2
seq_len = 20
display_step = 50
epoch = 1

In [7]:
class Model:
    def __init__(self, dim_word, dim_char, dropout, learning_rate,
                 hidden_size_char, hidden_size_word, num_layers):
        
        def cells(size, reuse=False):
            return tf.nn.rnn_cell.LSTMCell(size,initializer=tf.orthogonal_initializer(),reuse=reuse)
        
        def clip_grads(loss):
            variables = tf.trainable_variables()
            grads = tf.gradients(loss, variables)
            clipped_grads, _ = tf.clip_by_global_norm(grads, 5.0)
            return zip(clipped_grads, variables)
        
        self.word_ids = tf.placeholder(tf.int32, shape=[None, None])
        self.char_ids = tf.placeholder(tf.int32, shape=[None, None, None])
        self.labels = tf.placeholder(tf.int32, shape=[None, None])
        
        self.word_embeddings = tf.Variable(tf.truncated_normal([len(word2idx), dim_word],
                                                      stddev=1.0 / np.sqrt(dim_word)))
        self.char_embeddings = tf.Variable(tf.truncated_normal([len(char2idx), dim_char],
                                                      stddev=1.0 / np.sqrt(dim_char)))
        word_embedded = tf.nn.embedding_lookup(self.word_embeddings, self.word_ids)
        char_embedded = tf.nn.embedding_lookup(self.char_embeddings, self.char_ids)
        s = tf.shape(char_embedded)
        char_embedded = tf.reshape(char_embedded, shape=[s[0]*s[1], s[-2], dim_char])
        for n in range(num_layers):
            (out_fw, out_bw), (state_fw, state_bw) = tf.nn.bidirectional_dynamic_rnn(
                cell_fw = cells(hidden_size_char),
                cell_bw = cells(hidden_size_char),
                inputs = char_embedded,
                dtype = tf.float32,
                scope = 'bidirectional_rnn_char_%d'%(n))
            char_embedded = tf.concat((out_fw, out_bw), 2)
        output = tf.reshape(char_embedded[:,-1], shape=[s[0], s[1], 2*hidden_size_char])
        word_embedded = tf.concat([word_embedded, output], axis=-1)
        word_embedded = tf.nn.dropout(word_embedded, dropout)
        
        for n in range(num_layers):
            (out_fw, out_bw), (state_fw, state_bw) = tf.nn.bidirectional_dynamic_rnn(
                cell_fw = cells(hidden_size_word),
                cell_bw = cells(hidden_size_word),
                inputs = word_embedded,
                dtype=tf.float32,
                scope = 'bidirectional_rnn_word_%d'%(n))
            word_embedded = tf.concat((out_fw, out_bw), 2)
        word_embedded = tf.nn.dropout(word_embedded, dropout)
        
        W = tf.get_variable('w',shape=(2*hidden_size_word, len(idx2tag)),
                            initializer=tf.orthogonal_initializer())
        b = tf.get_variable('b',shape=(len(idx2tag)),initializer=tf.zeros_initializer())
        
        nsteps = tf.shape(word_embedded)[1]
        output = tf.reshape(word_embedded, [-1, 2*hidden_size_word])
        pred = tf.matmul(output, W) + b
        self.logits = tf.reshape(pred, [-1, nsteps, len(idx2tag)])
        
        log_likelihood, trans_params = tf.contrib.crf.crf_log_likelihood(
        self.logits, self.labels, tf.count_nonzero(self.word_ids, 1))
        
        self.cost = tf.reduce_mean(-log_likelihood)
        self.global_step = tf.Variable(0, trainable=False)
        
        self.learning_rate = tf.train.exponential_decay(learning_rate,
                                                        self.global_step, len(train_X) // batch_size,
                                                        0.1)
        
        self.crf_decode = tf.contrib.crf.crf_decode(self.logits, 
                                                    trans_params, 
                                                    tf.count_nonzero(self.word_ids, 1))[0]
        
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate).apply_gradients(clip_grads(self.cost), 
                                                                                    global_step=self.global_step)

In [8]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(dim_word,dim_char,dropout,learning_rate,hidden_size_char,hidden_size_word,num_layers)
sess.run(tf.global_variables_initializer())

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [9]:
def iter_seq(x):
    return np.array([x[i: i+seq_len] for i in range(0, len(x)-seq_len, 1)])

def to_train_seq(*args):
    return [iter_seq(x) for x in args]

def generate_char_seq(batch):
    x = [[len(idx2word[i]) for i in k] for k in batch]
    maxlen = max([j for i in x for j in i])
    temp = np.zeros((batch.shape[0],batch.shape[1],maxlen),dtype=np.int32)
    for i in range(batch.shape[0]):
        for k in range(batch.shape[1]):
            for no, c in enumerate(idx2word[batch[i,k]]):
                temp[i,k,-1-no] = char2idx[c]
    return temp

In [10]:
train_X_seq, train_Y_seq = to_train_seq(train_X, train_Y)
test_X_seq, test_Y_seq = to_train_seq(test_X, test_Y)

In [11]:
np.unique(train_Y_seq.ravel(),return_counts=True)

(array([1, 2, 3, 4, 5, 6, 7, 8]),
 array([ 198310, 2908994,   90686,  222459,  165511,     220,     740,
            480]))

In [12]:
np.unique(test_Y_seq.ravel(),return_counts=True)

(array([1, 2, 3, 4, 5, 7]),
 array([ 40886, 731743,  25207,  62733,  41871,     80]))

In [13]:
for i in range(epoch):
    total_cost = 0
    for k in range(0,(train_X_seq.shape[0] // batch_size)*batch_size,batch_size):
        batch_x = train_X_seq[k:k+batch_size]
        batch_y = train_Y_seq[k:k+batch_size]
        batch_x_char = generate_char_seq(batch_x)
        step, loss, _ = sess.run([model.global_step, model.cost, model.optimizer],
                                 feed_dict={model.word_ids:batch_x,
                                           model.char_ids:batch_x_char,
                                           model.labels:batch_y})
        if step % display_step == 0 or step == 1:
            print('epoch %d, step %d, loss %f'%(i+1,step,loss))
        total_cost += loss
    total_cost /= (train_X_seq.shape[0] // batch_size)
    print('epoch %d, avg loss %f'%(i+1,total_cost))

epoch 1, step 1, loss 43.618607
epoch 1, step 50, loss 12.042439
epoch 1, step 100, loss 27.015694
epoch 1, step 150, loss 19.762203
epoch 1, step 200, loss 8.836092
epoch 1, step 250, loss 12.508820
epoch 1, step 300, loss 6.588864
epoch 1, step 350, loss 7.509544
epoch 1, step 400, loss 3.870700
epoch 1, step 450, loss 8.295753
epoch 1, step 500, loss 12.325174
epoch 1, step 550, loss 4.284832
epoch 1, step 600, loss 4.885774
epoch 1, step 650, loss 2.738923
epoch 1, step 700, loss 8.448340
epoch 1, step 750, loss 1.698012
epoch 1, step 800, loss 7.499028
epoch 1, step 850, loss 3.572803
epoch 1, step 900, loss 10.391118
epoch 1, step 950, loss 1.823482
epoch 1, step 1000, loss 1.544621
epoch 1, step 1050, loss 4.242450
epoch 1, step 1100, loss 3.664677
epoch 1, step 1150, loss 7.973126
epoch 1, step 1200, loss 5.950489
epoch 1, step 1250, loss 6.812481
epoch 1, step 1300, loss 2.563293
epoch 1, step 1350, loss 1.441920
epoch 1, step 1400, loss 4.230231
epoch 1, step 1450, loss 4.453

In [15]:
label_Y, predicted_Y = [], []
for k in range(0,(test_X_seq.shape[0] // batch_size)*batch_size,batch_size):
    batch_x = test_X_seq[k:k+batch_size]
    batch_x_char = generate_char_seq(batch_x)
    batch_y = test_Y_seq[k:k+batch_size]
    Y_pred = sess.run(model.crf_decode,
                  feed_dict={model.word_ids:batch_x,
                            model.char_ids:batch_x_char})
    predicted_Y.append(Y_pred)
    label_Y.append(batch_y)

In [16]:
from sklearn.metrics import classification_report
print(classification_report(np.vstack(label_Y).ravel(), np.vstack(predicted_Y).ravel(), target_names=tag2idx.keys()))

             precision    recall  f1-score   support

        PAD       0.46      0.46      0.46     40883
      I-PER       0.96      0.99      0.97    731626
     B-MISC       0.68      0.25      0.37     25207
      I-ORG       0.79      0.64      0.71     62733
          O       0.75      0.77      0.76     41871
      B-ORG       0.00      0.00      0.00        80

avg / total       0.90      0.91      0.90    902400



/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 6, does not match size of target_names, 9
  .format(len(labels), len(target_names))
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
